In [64]:
import pandas as pd
import re

In [41]:
tweets_df = pd.read_csv('data/train.csv')

Data Preprocessing

In [42]:
tweets_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [43]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [44]:
tweets_df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [45]:
tweets_clean = tweets_df.copy()

In [47]:
tweets_clean['keyword'].value_counts()

keyword
unknown                61
fatalities             45
deluge                 42
armageddon             42
sinking                41
                       ..
forest fire            19
epicentre              12
threat                 11
inundation             10
radiation emergency     9
Name: count, Length: 222, dtype: int64

In [48]:
# Fill missing values and replace regex space
tweets_clean['keyword'] = tweets_clean['keyword'].fillna('unknown')
tweets_clean['keyword'] = tweets_clean['keyword'].apply(lambda x: x.replace('%20', ' '))

In [53]:
# Combine 'keyword' and 'text' into 'combined_text'
def combine_text(row):
    keyword = row['keyword']
    text = row['text']
    return f'keyword: {keyword} text: {text}'

tweets_clean['combined_text'] = tweets_clean.apply(combine_text, axis=1)

In [54]:
# Text cleaning
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text) # remove full mentions
    text = re.sub(r'#', '', text) # remove only hashtag sign but leave hashtag text
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

tweets_clean['combined_text'] = tweets_clean['combined_text'].apply(clean_text)

Define a Dataset Class